In [ ]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

In [ ]:
load_dotenv()  # take environment variables from .env.

import os


KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    file = file_loader.load()
    return file

In [ ]:
doc = read_doc("documents")

In [ ]:
len(doc)

In [ ]:
## Divide doc into chunks

def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc


In [ ]:
document = chunk_data(doc)

In [ ]:
len(document)

In [ ]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=KEY)


In [ ]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

In [ ]:
PINE_KEY = os.getenv("PINECONE_VECTOR")

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key='')
index_name = pc.Index("langchain")

In [ ]:
from langchain.vectorstores import Pinecone


In [ ]:
import os
os.environ['PINECONE_API_KEY'] = PINE_KEY

In [ ]:
index = pc.describe_index("langchain")['name']


In [ ]:
index=Pinecone.from_documents(doc,embeddings,index_name=index)


In [ ]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm=OpenAI(openai_api_key=KEY,temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [ ]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
our_query = "how many encodes are used?"
answer = retrieve_answers(our_query)
print(answer)